In [14]:
%matplotlib widget
import numpy as np
import igl
import rainbow.math.vector3 as V3
import rainbow.geometry.volume_mesh as VM
import rainbow.geometry.volume_mesh_wildmesh as WM
import rainbow.simulators.prox_soft_bodies.api as API
import rainbow.simulators.prox_soft_bodies.solver as SOLVER
import rainbow.util.viewer as VIEWER
from rainbow.util.USD import USD


In [25]:
# This block includes the helper functions to create the soft beam and the wall, and to initialize the viewer, and to simulate the scene.

# colors for visualization
COLOR_MAP = {
    'bunny': V3.make(0.1, 0.8, 0.1),
    'cube': V3.make(224/255, 208/255, 193/255),
    'ground': V3.make(224/255, 208/255, 193/255),
    'beam': V3.make(0.1, 0.8, 0.1),
    'teapot': V3.make(0.1, 0.8, 0.1)
}


# create a soft material into a scene
def create_soft_material(engine, name='soft_mat1'):
    if name == "soft_mat1":
        E, nu, rho = 1e5, 0.3, 1500
    else:
        E, nu, rho = API.create_material_parameters(None)
    API.create_material(engine, name)
    API.set_elasticity(engine, name, E, nu)
    API.set_mass_density(engine, name, rho)
    API.set_constitutive_model(engine, name, API.SNH)
    API.set_viscosity(engine, name, 0.5)
    API.create_surfaces_interaction(engine, name, name, 0.5)



# create the soft bunny into a scene
def create_soft_teapot(engine, material_name):
    V, F = igl.read_triangle_mesh("../data/teapot.obj") # geometry 
    rotation_mat = np.array([[1,0,0],[0,0,1],[0,-1,0]]) # rotate the teapot -90 degrees
    V = V.dot(rotation_mat.T)
    options = WM.Options()
    V_teapot, T_teapot = WM.create_mesh(V, F,options)
    V_teapot *= 2.0
    V_teapot[:,1] += 0.52
    y_min = np.min(V_teapot[:, 1])
    y_max = np.max(V_teapot[:, 1])
    print("Tube: y-min:{}, y-max:{}".format(y_min, y_max))
    API.create_soft_body(engine, 'teapot', V_teapot, T_teapot)
    API.set_type(engine, 'teapot', "Free")
    API.set_gravity(engine, 'teapot', (0,-50,0))
    API.set_material(engine,'teapot', material_name)

# create the wall into a scene
def create_cube(engine, material_name):
    V_wall, T_wall = VM.create_beam(20, 2, 10, 4, 0.05, 4)
    API.create_soft_body(engine, 'cube', V_wall, T_wall)
    API.set_type(engine, 'cube', "Free")
    API.set_gravity(engine, 'cube', (0,-50,0))
    API.set_material(engine,'cube', material_name)

# create the wall into a scene
def create_beam(engine, material_name):
    V_wall, T_wall = VM.create_beam(2, 2, 2, 0.5, 0.5, 0.5)
    V_wall[:,1] += 0.275
    API.create_soft_body(engine, 'beam', V_wall, T_wall)
    API.set_type(engine, 'beam', "Fixed")
    API.set_gravity(engine, 'beam', (0,-50,0))
    API.set_material(engine,'beam', material_name)

# create the ground into a scene
def create_ground(engine, material_name):
    V_wall, T_wall = VM.create_beam(2, 2, 2, 8.0, 0.1, 8.0)
    V_wall[:,1] -= 0.55
    y_min = np.min(V_wall[:, 1])
    y_max = np.max(V_wall[:, 1])
    print("Ground: y-min:{}, y-max:{}".format(y_min, y_max))
    API.create_soft_body(engine, 'ground', V_wall, T_wall)
    API.set_type(engine, 'ground', "Fixed")
    API.set_gravity(engine, 'ground', (0,0,0))
    API.set_material(engine,'ground', material_name)

# initialize the viewer
def initialize_viewer(engine, viewer, usd):
    # viewer.create_frame("origin")
    for body in engine.bodies.values():    
        opacity = 0.5 
        F = VM.to_triangles(body.T)
        color = COLOR_MAP[body.name]
        viewer.create_mesh(body.name, body.x0, F, color, opacity)   
        viewer.create_scatter(body.name + '.DBC')
        usd.add_mesh(body.name, body.x0, F)
    viewer.show()

# simulate the scene
def simulate(engine, viewer, usd):
    dt = engine.params.time_step
    T  = 0.45
    fps = 1.0/dt
    steps = int(np.round(T*fps))
    usd.set_animation_time(T*fps)
    for i in range(steps):
        print("simulation progressing--------------->{}/{}".format(i, steps))
        API.simulate(engine, dt, debug_on=True)
        for body in engine.bodies.values():
            viewer.update_mesh(body.name, body.x)
            usd.update_mesh_positions(body.name, body.x, i*dt*fps)
            if len(body.dirichlet_conditions)>0:
                indices = [bc.idx for bc in body.dirichlet_conditions]
                V = body.x[indices]
                viewer.update_scatter(body.name + '.DBC', V=V, scale=0.5, color=[0.4, 0.4, 0.4])
    return API.get_log(engine)

In [26]:


# scene1 setup
scene = API.create_engine()
scene.params.time_step = 0.001
viewer = VIEWER.Viewer()

create_soft_material(scene, "soft_mat1") 

create_beam(scene, 'soft_mat1')
create_cube(scene, 'soft_mat1')


# set the dirichlet conditions for the beam
def left_wall(x):
    return x[0] + 1.9

def bottom_ground(x):
    return x[2] + 0.4

# API.create_dirichlet_conditions(scene2, 'beam', left_wall)
# API.create_dirichlet_conditions(scene2, 'beam', bottom_ground)

# set the traction force for the beam
def top_cube(x):
    return 0.74 - x[1]

def bottom_beam(x):
    return x[1] + 0.4

def left_beam(x):
    return x[0] + 0.15

def top_beam(x):
    return 1.65 - x[1]

def center_cube(x):
    if (-0.5 <= x[0]  <= 0.5) and (-0.5 <= x[2]  <= 0.5):
        return -1
    else:
        return 1

def left_cube(x):
    return x[0] + 1.9

def right_cube(x):
    return 1.9 - x[0]



API.create_dirichlet_conditions(scene, 'cube', left_cube)
API.create_dirichlet_conditions(scene, 'cube', right_cube)
# API.create_traction_conditions(scene, 'cube', top_cube, load=V3.make(0, -2000,0))

usd = USD("soft_body_surface.usda")

initialize_viewer(scene, viewer, usd)
stats = simulate(scene, viewer, usd)

usd.save()

Renderer(camera=PerspectiveCamera(aspect=1.25, children=(DirectionalLight(color='white', intensity=0.6, positi…

simulation progressing--------------->0/450
simulation progressing--------------->1/450
simulation progressing--------------->2/450
simulation progressing--------------->3/450
simulation progressing--------------->4/450
simulation progressing--------------->5/450
simulation progressing--------------->6/450
simulation progressing--------------->7/450
simulation progressing--------------->8/450
simulation progressing--------------->9/450
simulation progressing--------------->10/450
simulation progressing--------------->11/450
simulation progressing--------------->12/450
simulation progressing--------------->13/450
simulation progressing--------------->14/450
simulation progressing--------------->15/450
simulation progressing--------------->16/450
simulation progressing--------------->17/450
simulation progressing--------------->18/450
simulation progressing--------------->19/450
simulation progressing--------------->20/450
simulation progressing--------------->21/450
simulation progressi